In [1]:
import pandas as pd
import numpy as np
from get_data import read_stock_data

In [2]:
def get_tencent_data(start_date, end_date):
    df = read_stock_data("data/00700")
    df['Date'] = df['date']
    df = df.set_index("Date")
    result = df.loc[start_date:end_date]
    return result


# 计算EMA
def ema(data, n):
    return data.ewm(span=n, adjust=False).mean()

# 计算MACD指标
def macd(data):
    data['DIF'] = ema(data['close'], 12) - ema(data['close'], 26)
    data['DEA'] = ema(data['DIF'], 9)
    data['MACD'] = 2 * (data['DIF'] - data['DEA'])
    return data

# 回测MACD策略
def backtest_macd(data):
    data['Signal'] = np.where(data['DIF'] > data['DEA'], 1, -1)
    data['Return'] = data['close'].pct_change() * data['Signal'].shift(1)
    data['Cumulative_Return'] = (1 + data['Return']).cumprod()
    return data

In [3]:
# 获取腾讯科技的历史数据
start_date = '2023-01-01'
end_date = '2023-04-15'
tencent_data = get_tencent_data(start_date, end_date)

    

In [4]:
# 计算MACD指标
tencent_data = macd(tencent_data)

%whos

Variable           Type         Data/Info
-----------------------------------------
backtest_macd      function     <function backtest_macd at 0x7fe6e1a14550>
ema                function     <function ema at 0x7fe6e1a14430>
end_date           str          2023-04-15
get_tencent_data   function     <function get_tencent_data at 0x7fe6e1a143a0>
macd               function     <function macd at 0x7fe6e1a144c0>
np                 module       <module 'numpy' from '/Us<...>kages/numpy/__init__.py'>
pd                 module       <module 'pandas' from '/U<...>ages/pandas/__init__.py'>
read_stock_data    function     <function read_stock_data at 0x7fe6a0186ee0>
start_date         str          2023-01-01
tencent_data       DataFrame                Unnamed: 0   <...>n\n[68 rows x 10 columns]


In [5]:
# # 回测MACD策略
tencent_data = backtest_macd(tencent_data)



In [6]:
# 输出策略收益
print("MACD策略的累计收益：", tencent_data['Cumulative_Return'].iloc[-1])
    

MACD策略的累计收益： 1.2005967166712295


In [7]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(tencent_data[['date', 'close', 'DIF', 'DEA', 'Signal', 'MACD', 'Return']])


                  date     close        DIF        DEA  Signal       MACD   
Date                                                                        
2023-01-03  2023-01-03  327.8635   0.000000   0.000000      -1   0.000000  \
2023-01-04  2023-01-04  342.8700   1.197100   0.239420       1   1.915360   
2023-01-05  2023-01-05  347.8000   2.514633   0.694462       1   3.640340   
2023-01-06  2023-01-06  349.4000   3.645866   1.284743       1   4.722246   
2023-01-09  2023-01-09  362.0000   5.495740   2.126943       1   6.737595   
2023-01-10  2023-01-10  362.4000   6.914351   3.084424       1   7.659853   
2023-01-11  2023-01-11  373.8000   8.856404   4.238820       1   9.235167   
2023-01-12  2023-01-12  364.0000   9.495261   5.290108       1   8.410306   
2023-01-13  2023-01-13  371.4000  10.477896   6.327666       1   8.300460   
2023-01-16  2023-01-16  370.2000  11.032633   7.268659       1   7.527948   
2023-01-17  2023-01-17  373.8000  11.628708   8.140669       1   6.976078   

In [8]:
print(len(tencent_data))
print(tencent_data.loc["2023-01-18", "high"])
print(tencent_data.iloc[4, 3])
print(tencent_data["open"][2])


68
381.0
362.4
358.0
